In [ ]:
import os
import re
import sys
import argparse
import subprocess
import matplotlib
import time
import pandas as pd
import numpy as np
from itertools import groupby
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from pylab import *
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
import csv
from collections import defaultdict
import operator
import joblib
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from Bio import SeqIO
from sklearn.feature_selection import RFE
from sklearn import preprocessing
import pybedtools as pbt
import pyBigWig as pbw
import glob
from datetime import date
import random
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from itertools import chain
import string
import shlex
import shutil
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from sklearn.ensemble import AdaBoostClassifier
from adjustText import adjust_text
import matplotlib.ticker as ticker


In [ ]:
os.chdir("...") 
root = "..." 
basicdir = os.path.abspath('GHTS/') 
outputdir = os.path.abspath('GHTS/outputdir')
train_dir = os.path.abspath('GHTS/Train/') 
test_dir = os.path.abspath('GHTS/Test/') 

if not os.path.exists(basicdir): 
    os.makedirs(basicdir)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)


In [ ]:
TFs_CHS_AFS = pd.read_csv("....txt", sep="\t", header=None)
TFs_CHS_AFS

In [ ]:
fasta = pbt.example_filename('../hg38.fa')

In [ ]:
if not os.path.exists("../pbt_exclusion_zone"):
    os.makedirs("../pbt_exclusion_zone")
pbt.helpers.set_tempdir("../pbt_exclusion_zone")

In [ ]:
for i in [x for x in os.listdir(".../Test/") if x in list(TFs_CHS_AFS[0])]:
    print(test_dir+"/"+i)
    if not os.path.exists(test_dir+"/"+i):
        os.makedirs(test_dir+"/"+i)
    for j in os.listdir(".../Test/"+i):
        #print(j)
        dfm = pd.read_csv(f".../Test/{i}/foreigns.bed", sep="\t", header=None)
        foreigns = pbt.BedTool.from_dataframe(dfm)
        foreigns = foreigns.sequence(fi=fasta)
        foreigns_fa = open(foreigns.seqfn).read()
        with open(test_dir+"/"+i+"/foreigns.fa", "w") as f:
            f.write(foreigns_fa)
            
        dfm = pd.read_csv(f".../Test/{i}/positives.bed", sep="\t", header=None)
        positives = pbt.BedTool.from_dataframe(dfm)
        positives = positives.sequence(fi=fasta)
        positives_fa = open(positives.seqfn).read()
        with open(test_dir+"/"+i+"/positives.fa", "w") as f:
            f.write(positives_fa)
        
        dfm = pd.read_csv(f".../Test/{i}/random.bed", sep="\t", header=None)
        random = pbt.BedTool.from_dataframe(dfm)
        random = random.sequence(fi=fasta)
        random_fa = open(random.seqfn).read()
        with open(test_dir+"/"+i+"/random.fa", "w") as f:
            f.write(random_fa)
        
        dfm = pd.read_csv(f"../Test/{i}/shades.bed", sep="\t", header=None)
        shades = pbt.BedTool.from_dataframe(dfm)
        shades = shades.sequence(fi=fasta)
        shades_fa = open(shades.seqfn).read()
        with open(test_dir+"/"+i+"/shades.fa", "w") as f:
            f.write(shades_fa)

In [ ]:
for i in [x for x in os.listdir(".../Train/") if x in list(TFs_CHS_AFS[0])]:
    print(train_dir+"/"+i)
    if not os.path.exists(train_dir+"/"+i): # если нет result, то создаем
        os.makedirs(train_dir+"/"+i)
    for j in os.listdir(".../Train/"+i):
        #print(j)
        dfm = pd.read_csv(f".../Train/{i}/foreigns.bed", sep="\t", header=None)
        foreigns = pbt.BedTool.from_dataframe(dfm)
        foreigns = foreigns.sequence(fi=fasta)
        foreigns_fa = open(foreigns.seqfn).read()
        with open(train_dir+"/"+i+"/foreigns.fa", "w") as f:
            f.write(foreigns_fa)
            
        dfm = pd.read_csv(f".../Train/{i}/positives.bed", sep="\t", header=None)
        positives = pbt.BedTool.from_dataframe(dfm)
        positives = positives.sequence(fi=fasta)
        positives_fa = open(positives.seqfn).read()
        with open(train_dir+"/"+i+"/positives.fa", "w") as f:
            f.write(positives_fa)
        
        dfm = pd.read_csv(f".../Train/{i}/random_addshift.bed", sep="\t", header=None)
        random = pbt.BedTool.from_dataframe(dfm)
        random = random.sequence(fi=fasta)
        random_fa = open(random.seqfn).read()
        with open(train_dir+"/"+i+"/random.fa", "w") as f:
            f.write(random_fa)
        
        dfm = pd.read_csv(f".../GHTS/Train/{i}/shades_addshift.bed", sep="\t", header=None)
        shades = pbt.BedTool.from_dataframe(dfm)
        shades = shades.sequence(fi=fasta)
        shades_fa = open(shades.seqfn).read()
        with open(train_dir+"/"+i+"/shades.fa", "w") as f:
            f.write(shades_fa)

In [ ]:
for i in [x for x in os.listdir(".../Test/") if x in list(TFs_CHS_AFS[0])]:
    print(test_dir+"/"+i)
    if not os.path.exists(test_dir+"/"+i): # если нет result, то создаем
        os.makedirs(test_dir+"/"+i)
    for j in os.listdir(".../Test/"+i):

        line = f"awk 'NF' {test_dir}/{i}/positives.fa > {test_dir}/{i}/positives_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()
        line = f"awk 'NF' {test_dir}/{i}/foreigns.fa > {test_dir}/{i}/foreigns_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()
        
        line = f"awk 'NF' {test_dir}/{i}/random.fa > {test_dir}/{i}/random_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()

        line = f"awk 'NF' {test_dir}/{i}/shades.fa > {test_dir}/{i}/shades_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()

In [ ]:
for i in [x for x in os.listdir(".../Train/") if x in list(TFs_CHS_AFS[0])]:
    print(train_dir+"/"+i)
    if not os.path.exists(train_dir+"/"+i): # если нет result, то создаем
        os.makedirs(train_dir+"/"+i)
    for j in os.listdir(".../Train/"+i):

        line = f"awk 'NF' {train_dir}/{i}/positives.fa > {train_dir}/{i}/positives_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()
        line = f"awk 'NF' {train_dir}/{i}/foreigns.fa > {train_dir}/{i}/foreigns_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()
        
        line = f"awk 'NF' {train_dir}/{i}/random.fa > {train_dir}/{i}/random_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()

        line = f"awk 'NF' {train_dir}/{i}/shades.fa > {train_dir}/{i}/shades_no_NF.fa"
        print(line)
        p = subprocess.Popen(line, shell=True)
        p.wait()